# Notes
- Preference is given to use PV avaiable in the current timestep.
- If there is excess PV energy available in the current timestep, it is stored in the battery.
- Battery is topped up with grid electricity from the cheapest hour before the next peak hour, so that the battery is full going into the next peak hour.



In [1]:
import pandas as pd
import numpy as np
import random
from statistics import mean
import matplotlib.pyplot as plt
import copy
import math

# Parameters

In [2]:
start_date = '2022-01-01 00:00:00'
end_date = '2022-12-31 00:00:00'
battery_cap = 500  # in kWh
battery_charge_discharge_rate = 250  # in kW
initial_soc = 0 # in kWh, the state of charge in the battery at the start of the time horizon
charge_discharge_percentage = random.uniform(0.5, 1)

# Data Preprocessing

In [3]:
df_prediction = pd.read_csv('Vyskov.csv')
df_consumption = pd.read_csv('historie-spotreb-elektrina.csv')
column_renames = {"Datum a čas": "Timestamp",
                  "Činná spotřeba hodinová (kWh)": "Hourly consumption (kWh)",
                  "Jalová spotřeba při spotřebě hodinová (kVAR)": "Reactive hourly consumption (kVAR)",
                  "Jalová dodávka při spotřebě hodinová (kVAR)": "Reactive supply hourly consumption (kVAR)"
                  }
df_consumption = df_consumption.rename(columns=column_renames)

prediction_column_rename = {"time_utc": "Timestamp", "production_prediction": "PV Energy Prediction (kWh)"}

df_prediction = df_prediction.rename(columns = prediction_column_rename)

df_consumption['Timestamp'] = pd.to_datetime(df_consumption['Timestamp'], format='%d.%m.%Y %H:%M')
df_prediction['Timestamp'] = pd.to_datetime(df_prediction['Timestamp'], format='%d/%m/%Y %H:%M')

In [4]:
df_consumption = df_consumption[["Timestamp", "Hourly consumption (kWh)"]]
df_prediction = df_prediction[["Timestamp", "PV Energy Prediction (kWh)"]]


df_combined = pd.merge(df_consumption, df_prediction, on="Timestamp")
df_combined.set_index('Timestamp', inplace=True)

In [5]:
# Define TOU rates for 24 hours (hour 0 = 0.23, hour 1 = 0.28, etc.)
tou_rates = {
    0: 0.23, 1: 0.28, 2: 0.27, 3: 0.25, 4: 0.22, 5: 0.20,   # 12 AM - 6 AM
    6: 0.30, 7: 0.35, 8: 0.40, 9: 0.45, 10: 0.42, 11: 0.38, # 6 AM - 12 PM
    12: 0.46, 13: 0.44, 14: 0.52, 15: 0.53, 16: 0.47, 17: 0.68, # 12 PM - 6 PM
    18: 0.64, 19: 0.50, 20: 0.45, 21: 0.35, 22: 0.29, 23: 0.22  # 6 PM - 12 AM
}


In [6]:
# Function to slice the DataFrame based on the start and end date
def get_time_horizon(df, start_date, end_date):
    # Ensure start_date and end_date are datetime objects
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    # Slice the DataFrame using the loc function
    sliced_df = df.loc[start_date:end_date].copy()
    
    return sliced_df

# Get the sliced DataFrame for the defined time horizon
df = get_time_horizon(df_combined, start_date, end_date)

# Extract the hour and assign TOU rates using .loc to avoid the SettingWithCopyWarning
df.loc[:, 'hour'] = df.index.hour
df.loc[:, 'TOU (€/kWh)'] = df['hour'].map(tou_rates)

# Drop the 'hour' column as it's no longer needed
df.drop(columns=['hour'], inplace=True)

df.to_csv('Input_df_03_06_2022_08_03_2022')
df

,Hourly consumption (kWh),PV Energy Prediction (kWh),TOU (€/kWh)
Timestamp,,,
2022-01-01 00:00:00,22,0.000607,0.23
2022-01-01 01:00:00,21,0.000607,0.28
2022-01-01 02:00:00,23,0.000607,0.27
2022-01-01 03:00:00,21,0.000607,0.25
2022-01-01 04:00:00,21,0.000607,0.22
...,...,...,...
2022-12-30 20:00:00,20,0.000607,0.45
2022-12-30 21:00:00,21,0.000607,0.35
2022-12-30 22:00:00,20,0.000607,0.29


# Greedy Algorithm

In [7]:
def init_schedule(df):
    discharging = False
    schedule = []
    soc = initial_soc
    peak_hour_tracker = 0
    num_peak_hours = (df['TOU (€/kWh)'] >= 0.6).sum()
    next_peak_hour = df[df['TOU (€/kWh)'] >= 0.6].index[peak_hour_tracker]
    expensive_tou_rates = None
    
    for t in range(len(df)):
        timestamp = df.index[t]
        hourly_consumption = df['Hourly consumption (kWh)'][t]
        pv_available = df['PV Energy Prediction (kWh)'][t]
        tou_rate = df['TOU (€/kWh)'][t]
        battery_change_not_pv = 0
        if tou_rate >= 0.6:
            discharging = True
        
        if t == 0:
            previous_timestamp = None
            previous_tou = None
            previous_soc = initial_soc
        else:
            previous_timestamp = df.index[t-1]
            previous_tou = df['TOU (€/kWh)'][t-1]
            previous_soc = schedule[t-1][3]
        
        # USE ALL PV DURING THIS TIMESTEP, STORE EXCESS IN BATTERY
        pv_use_now = min(pv_available, hourly_consumption)
        excess_pv = max(0, (hourly_consumption - pv_available))
        pv_into_battery = min(battery_charge_discharge_rate, battery_cap - soc, excess_pv)
        
        # Update soc with this pv energy into the battery
        soc += pv_into_battery
        
        
        # TOP-UP THE BATTERY WITH GRID ELECTRICITY IF THERE IS ROOM TO DO SO
        # This block of code finds the cheapest hours to charge the battery during the charging period, i.e.
        # the period before the next peak tou rate
        if next_peak_hour and tou_rate < 0.6:
            # Now in charging period, i.e. want all PV to battery and topped up with cheap grid electricity
            charge_df = df[(df.index >= timestamp) & (df.index < next_peak_hour)]
            unique_tou_rates = charge_df['TOU (€/kWh)'].unique()
            ordered_tou_rates = sorted(unique_tou_rates)
            
            # Sum up PV into the battery before the peak hour
            pv_sum_before_peak = 0
            soc_before_peak = soc  # Temporary variable to track SOC before the peak hour
            for _, row in charge_df.iterrows():
                pv_available_summing = row['PV Energy Prediction (kWh)']
                # Calculate the amount of PV that can go into the battery
                excess_pv = min(0, (hourly_consumption - pv_available_summing))
                pv_into_battery_this_hour = min(battery_charge_discharge_rate, battery_cap - soc_before_peak, excess_pv)
                # Accumulate the PV that goes into the battery
                pv_sum_before_peak += pv_into_battery_this_hour
                # Update the state of charge after this hour
                soc_before_peak += pv_into_battery_this_hour
            
            
            
            battery_space = max(0, (battery_cap - pv_sum_before_peak - soc))
            num_hours_top_up = math.ceil(battery_space / battery_charge_discharge_rate)
            cheapest_tou_rates = ordered_tou_rates[:num_hours_top_up]
        
        # If the tou_rate is in the cheapest tou rates for this period, charge battery with grid electricity.
        # If there is no next_peak_hour (i.e. next_peak_hour = None), dont charge the battery with grid electricity
        if next_peak_hour and (tou_rate in cheapest_tou_rates):
            fill_up_space_percentage = random.uniform(1, 1)
            charge_rate = min(battery_space, battery_charge_discharge_rate - pv_into_battery, battery_cap - soc)
            battery_change_not_pv = charge_rate * fill_up_space_percentage
            battery_space -= battery_change_not_pv
        
                
        # Update the next_peak_hour
        if tou_rate >= 0.6:
            peak_hour_tracker += 1
            if peak_hour_tracker == num_peak_hours:
                next_peak_hour = None
            else:
                next_peak_hour = df[df['TOU (€/kWh)'] >= 0.6].index[peak_hour_tracker]
                
                
        # DISCHARGE THE BATTERY
        # Define the hours to discharge the battery in
        if tou_rate >= 0.6:
            if next_peak_hour:
                discharge_df = df[(df.index >= timestamp) & (df.index < next_peak_hour)]
                #print(f"Start time for discharge_df = {timestamp}, End time = {next_peak_hour}")
            else:
                discharge_df = df[(df.index >= timestamp) & (df.index <= end_date)]
                #print(f"Start time for discharge_df = {timestamp}, End time = {end_date}")
            unique_discharge_tou_rates = discharge_df['TOU (€/kWh)'].unique()
            ordered_discharge_tou_rates = sorted(unique_discharge_tou_rates, reverse = True)
            
            # Define the number of hours that the battery can discharge at maximum capacity
            # Added 2 to make up for the PV energy entering the battery during the hours where
            # tou <= 0.6. This makes sure the battery will be fully discharged.
            num_hours_discharge = math.ceil(soc / battery_charge_discharge_rate) + 2
            expensive_tou_rates = ordered_discharge_tou_rates[:num_hours_discharge]
        
        # If the tou_rate is in these hours, discharge the battery at the maximum rate
        if discharging == True and expensive_tou_rates and (tou_rate in expensive_tou_rates):
            excess_hourly_consumption = max(0, hourly_consumption - pv_use_now)
            discharge_rate = max(-battery_charge_discharge_rate, -soc, -excess_hourly_consumption)
            discharge_percentage = random.uniform(1, 1)
            battery_change_not_pv += discharge_rate * discharge_percentage


        
        
        # Update the soc with the battery change that is not pv
        soc += battery_change_not_pv
        if soc < 1:
            discharging = False
            expensive_tou_rates = []
        elif soc > battery_cap * 0.99:
            discharging = True
        
        
        # Update the battery change
        battery_change = battery_change_not_pv + pv_into_battery
        
        
        grid_usage = max(0, hourly_consumption - pv_use_now + battery_change)
        
        schedule.append((grid_usage, pv_use_now, battery_change, soc, t, pv_into_battery, battery_change_not_pv))
        
                
        if soc < 0 or soc > battery_cap:
            print(f"ERROR: SOC OUT OF BOUNDS")
            print(f"Previous Soc = {previous_soc:.3f}, battery_change_not_PV = {battery_change_not_pv:.3f}, pv into battery = {pv_into_battery:.3f}, battery change = {battery_change:.3f}, current soc = {soc:.3f}")
            
        
        if abs(soc - previous_soc - battery_change) > 1e-3:
            print(f"ERROR IN INITIALISATION FUNCTION, PREVIOUS SOC + BATTERY CHANGE != CURRENT SOC")
            difference = soc - previous_soc - battery_change
            print(f" Hour = {t}, Previous SOC = {previous_soc}, Battery Change = {battery_change}, Current SOC = {soc}")
            print(f"difference = {difference}")
        
        #print(f"Hour = {t}, tou rate = {tou_rate}, hourly consumption = {hourly_consumption}, Grid usage = {grid_usage}, PV used = {pv_use_now}, battery change = {battery_change}")
        if abs(hourly_consumption - (grid_usage + pv_use_now - battery_change)) > 1e-3:
            print("ERROR IN INITIALISATION FUNCTION, ENERGY IMBALANCE")
            print(f"Hour = {t}, SOC = {soc}, pv into battery = {pv_into_battery} bat change not pv = {battery_change_not_pv}, Excess hourly consumption = {excess_hourly_consumption}, tou rate = {tou_rate}, hourly consumption = {hourly_consumption}, Grid usage = {grid_usage}, PV used = {pv_use_now}, battery change = {battery_change}")
            
            
        if battery_change > battery_charge_discharge_rate:
            print(f"ERROR IN INITIALISATION: BATTERY CHANGE > BATTERY CHARGE LIMIT. BATTERY CHANGE = {battery_change}")
    return schedule        

In [8]:
# Fitness function to evaluate a schedule
def schedule_fitness(schedule, df):
    tou_rates = df['TOU (€/kWh)'].values
    total_cost = 0
    for (grid_usage, pv_used, battery_change, soc, hour, pv_into_battery, battery_change_not_pv) in schedule:
        total_cost += grid_usage * tou_rates[hour]
    return total_cost

In [9]:
greedy_schedule = init_schedule(df)

In [10]:
schedule_price = schedule_fitness(greedy_schedule, df)
print(f"Greedy Schedule Price is {schedule_price}")

Greedy Schedule Price is 657504.3964636596


In [11]:
def save_greedy_schedule_to_csv(greedy_schedule, original_df, filename):
    # Create a pandas DataFrame from the best schedule
    df = pd.DataFrame(greedy_schedule, columns=['Grid Usage (kWh)', 'PV Used during this timestep (kWh)', 'Battery Charge/Discharge (kWh)', 'Battery State of Charge (kWh)', 'Hour', 'PV into Battery (kWh)', 'Battery Change not including PV (kWh)'])
    
    # Drop the 'Hour' column as it is not required in the CSV file
    df = df.drop(columns=['Hour'])
    df['Timestamp'] = original_df.index
    df.set_index('Timestamp', inplace=True)
    df['Hourly Consumption (kWh)'] = original_df['Hourly consumption (kWh)']
    df['TOU rate (€/kWh)'] = original_df['TOU (€/kWh)']
    df['Hourly Cost (€)'] = df['TOU rate (€/kWh)'] * df['Grid Usage (kWh)']
    df['Energy Balance'] = df['Hourly Consumption (kWh)'] + df['Battery Charge/Discharge (kWh)'] - df['Grid Usage (kWh)'] - df['PV Used during this timestep (kWh)']
    df['Energy Balance'] = df['Energy Balance'].round(4)
    
    # Save the DataFrame to a CSV file
    df.to_csv(filename)

In [12]:
#save_greedy_schedule_to_csv(greedy_schedule, df, 'Sample_1_GREEDY_2.0_SPOT_TOU_03_06_2022_08_06_2022_500kWh.csv')